In [5]:
import sys
sys.executable

'/Library/Developer/CommandLineTools/usr/bin/python3'

In [9]:
!/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-24.0-py3-none-any.whl (2.1 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
!/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install scipy matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl (35.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 1.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 18.0 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm
from scipy.optimize import minimize

In [12]:
class Interest:
    def __init__(self,
                 pv: float,
                 fv: float,
                 r: float,
                 n: int,
                 m: int,
                 a: float):
        self.pv = pv # present value
        self.fv = fv # future value
        self.r = r # interest rate
        self.n = n # number of period
        self.m = m # number of sub-period per period
        self.a = a # sub-period periodic deposit

    def fv_simple(pv: float, r: float, n: int):
        return (1+r*n)*pv
    
    def pv_simple(fv: float, r: float, n: int):
        return fv / (1 + r * n)
        
    def r_simple(pv: float, fv: float, n: int):
        return (fv/pv - 1) /  n
    
    def fv_compound(pv, r: float, n: int):
        return pv * (1+r)**n
    
    def pv_compound(fv: float, r: float, n: int):
        return fv / (1 + r)**n
    
    def r_compound(pv: float, fv: float, n: int):
        return (fv/pv)**(1/n) - 1
    
    def fv_interval(pv: float, r: float, n: int, m: int):
        return ((1 + r / m)**(m * n)) * pv
    
    def pv_interval(fv: float, r: float, n: int, m: int):
        return fv / (1+r/m)**(m*n)
    
    def r_interval(pv: float, fv: float, n: int, m: int):
        return ((fv / pv)**(1/(m*n)) - 1)* m
    
    def fv_continuous(pv: float, r: float, n: int):
        return np.exp(r * n) * pv
    
    def pv_continuous(fv: float, r: float, n: int):
        return fv / np.exp(r * n)
    
    def r_continuous(pv: float, fv: float, n: int):
        return np.log(fv/pv) / n
    
    def fv_with_deposit(r: float, n: int, m: int, a: float):
        return (a / (r/m)) * ((1 + r/m)**(m*n) - 1)
    
    def pv_with_deposit(r: float, n: int, m: int, a: float):
        return (a / (r/m)) * (1 - 1 / (1 + r / m)**(m*n))

In [15]:
class BlackScholes:
    
    def __init__(self,
                 S: float,
                 K: float,
                 T: int,
                 r: float,
                 sigma: float,
                 price: float,
                 q: float):
        self.S = S # Price of underlying stock
        self.K = K # Strike price
        self.T = T # Time till expiration (in years)
        self.r = r # Risk-free interest rate (0.05 indicates 5%)
        self.sigma = sigma # Volatility (standard deviation) of stock (0.15 indicates 15%)
        self.price = price # option price to calculate implied volatility
        self.q = q # Annual dividend yield (0.05 indicates 5%)
    
    @staticmethod
    def _d1(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        return (1 / (sigma * np.sqrt(T))) * (np.log(S/K) + (r + sigma**2 / 2) * T)
    
    def _d1(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        return (1 / (sigma * np.sqrt(T))) * (np.log(S * np.exp(-q*T) / K) + (r + sigma**2 / 2) * T)
    
    def _d2(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        return self._d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    def _d2(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        return self._d1(S, K, T, r, sigma, q) - sigma * np.sqrt(T)
    
    def call_price(self, S=None, K=None, T=None, r=None, sigma=None):
        """ Main method for calculating price of a call option """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r*T)
    
    def call_price(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        """ Main method for calculating price of a dividend call option """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d1 = self._d1(S, K, T, r, sigma, q)
        d2 = self._d2(S, K, T, r, sigma, q)
        return norm.cdf(d1) * S * np.exp(-q*T) - norm.cdf(d2) * K * np.exp(-r*T)
    
    def put_price(self, S=None, K=None, T=None, r=None, sigma=None):
        """ Main method for calculating price of a put option """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(-d2) * K * np.exp(-r*T) - norm.cdf(-d1) * S
    
    def put_price(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        """ Main method for calculating price of a dividend put option """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d1 = self._d1(S, K, T, r, sigma, q)
        d2 = self._d2(S, K, T, r, sigma, q)
        return norm.cdf(-d2) * K * np.exp(-r*T) - norm.cdf(-d1) * S * np.exp(-q*T)
    
    def call_in_the_money(self, S=None, K=None, T=None, r=None, sigma=None):
        """ 
        Calculate probability that call option will be in the money at
        maturity according to Black-Scholes.
        """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d2 = self._d2(self, S, K, T, r, sigma)
        return norm.cdf(d2)
    
    def call_in_the_money(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        """ 
        Calculate probability that dividend call option will be in the money at
        maturity according to Black-Scholes.
        """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d2 = self._d2(S, K, T, r, sigma, q)
        return norm.cdf(d2)
    
    def put_in_the_money(self, S=None, K=None, T=None, r=None, sigma=None):
        """ 
        Calculate probability that put option will be in the money at
        maturity according to Black-Scholes.
        """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d2 = self._d2(self, S, K, T, r, sigma)
        return 1 - norm.cdf(d2)
    
    def put_in_the_money(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        """ 
        Calculate probability that dividend put option will be in the money at
        maturity according to Black-Scholes.
        """
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d2 = self._d2(S, K, T, r, sigma, q)
        return 1 - norm.cdf(d2)
    
    def call_implied_volatility(self, price=None, S=None, K=None, T=None, r=None):
        """ Calculate implied volatility of a call option up to 2 decimals of precision. """
        if price is None: price = self.price
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        sigma = 0.0001
        while sigma < 1:
            d1 = BlackScholes()._d1(S, K, T, r, sigma)
            d2 = BlackScholes()._d2(S, K, T, r, sigma)
            price_implied = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
            if price - price_implied < 0.0001:
                return sigma
            sigma += 0.0001
        return "Not Found"
    
    def put_implied_volatility(self, price=None, S=None, K=None, T=None, r=None):
        """ Calculate implied volatility of a put option up to 2 decimals of precision. """
        if price is None: price = self.price
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        sigma = 0.0001
        while sigma < 1:
            call = BlackScholes().call_price(S, K, T, r, sigma)
            price_implied = K * np.exp(-r*T) - S + call
            if price - price_implied < 0.0001:
                return sigma
            sigma += 0.0001
        return "Not Found"
    
    # following methods calculates european put/call option greeks
    
    def call_delta(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return norm.cdf(d1)
    
    def call_gamma(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    def call_vega(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return S * norm.pdf(d1) * np.sqrt(T)
    
    def call_theta(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return - ((S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T))) - r * K * np.exp(-r*T) * norm.cdf(d2)
    
    def call_rho(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d2 = self._d2(S, K, T, r, sigma)
        return K * T * np.exp(-r*T) * norm.cdf(d2)
    
    def call_epsilon(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d1 = self._d1(S, K, T, r, sigma)
        return -S * T * np.exp(-q*T) * norm.cdf(d1)
    
    def put_delta(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return norm.cdf(d1) - 1
    
    def put_gamma(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return norm.pdf(d1) / (S * sigma * np.sqrt(T))
    
    def put_vega(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        return S * norm.pdf(d1) * np.sqrt(T)
    
    def put_theta(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return - ((S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T))) + r * K * np.exp(-r*T) * norm.cdf(-d2)
    
    def put_rho(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d2 = self._d2(S, K, T, r, sigma)
        return - K * T * np.exp(-r*T) * norm.cdf(-d2)
    
    def put_epsilon(self, S=None, K=None, T=None, r=None, sigma=None, q=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        if q is None: q = self.q
        d1 = self._d1(S, K, T, r, sigma)
        return S * T * np.exp(-q*T) * norm.cdf(-d1)
    
    def vanna(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.pdf(d1) * d2 / sigma
    
    def charm(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return -norm.pdf(d1) * ((2 * r * T - d2 * sigma * np.sqrt(T)) / (2 * T * sigma * np.sqrt(T)))
    
    def vomma(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return S * norm.pdf(d1) * np.sqrt(T) * ((d1 * d2) / sigma)
    
    def veta(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return -S * norm.pdf(d1) * np.sqrt(T) * (r * d1 / (sigma * np.sqrt(T)) - (1 + d1 * d2) / (2 * T))
    
    def zomma(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.pdf(d1) * (d1*d2 - 1) / (S * sigma**2 * np.sqrt(T))
    
    def color(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.pdf(d1) / (2*S*T*sigma*np.sqrt(T)) * (2*T + 1 + ((2*r*T - d2*sigma*np.sqrt(T)) / sigma*np.sqrt(T)) * d1)
    
    def ultima(self, S=None, K=None, T=None, r=None, sigma=None):
        if S is None: S = self.S
        if K is None: K = self.K
        if T is None: T = self.T
        if r is None: r = self.r
        if sigma is None: sigma = self.sigma
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        vega = self.call_vega(S, K, T, r, sigma)
        return -vega/sigma**2 * (d1 * d2 * (1-d1*d2) + d1**2 + d2**2)

In [14]:
class OptionStrategies:
    @staticmethod
    def short_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return - put - call
    
    @staticmethod
    def long_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return - put - call

    @staticmethod
    def long_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_put_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        put3 = BlackScholes().put_price(S, K3, T, r, sigma)
        return - put1 + 2 * put2 - put3
    
    @staticmethod
    def long_call_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        call1 = BlackScholes().call_price(S, K1, T, r, sigma)
        call2 = BlackScholes().call_price(S, K2, T, r, sigma)
        call3 = BlackScholes().call_price(S, K3, T, r, sigma)
        return call1 - 2 * call2 + call3
    
    @staticmethod
    def short_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)
        return put1 - put2 - call1 + call2
    
    @staticmethod
    def long_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)
        return - put1 + put2 + call1 - call2